### 天梯最终版：天梯和原来相比，第二版变化在哪里——
- 改变了delta的计算公式，引入了compensate以及compensate的参数b，b的取值由未经compensate的结果计算得到。
    - 其中duration的计算公式也进行了微调。微调的部分在于，当correct = 0时，我们应该对做得又慢又错的人加以最大的惩罚，对做得错但是很快的人加以较小的惩罚，因而correct=0时的norm_duration()部分我进行了对应的处理。
- 取出数据的时候先按照day和event_time进行了排序，以便后面trueskill计算时可以按照user的答题提交顺序来进行。因为除了胜负总次数之外，胜负的顺序对于trueskill的计算也有非常大的影响。除了trueskill之外，还有ts_pts也要按时间顺序计算。
- 改变了总分的计算方式，没有参照F1积分法去计算。重新设计了一个新函数去将rank映射到积分。
     - 这样做的原因是，当学校数量不只10个的时候，由于F1积分只有10名，所以明显不合适。所以设计了一个连续函数，而且在rank靠前（名次较高）的时候对于积分有更强的区分度，即该函数是二阶导小于零的单调减函数。这样可以将那些大部分知识点排名都很靠前优秀学校们更好地区分开（而且这样看起来也更符合F1积分的思想）
- 对于个人贡献度的计算：采取了百分数的统计方式。即“您的贡献值已经超过30%的校友”类似的。


In [1]:
from impala.dbapi import connect
import pandas as pd
from collections import defaultdict
import trueskill as ts
import multiprocessing as mps
import numpy as np

### 连接HUE数据库

In [2]:
conn = connect(host='10.8.8.21', port=10015, auth_mechanism='PLAIN', database='tmp', user='shenfei',password='123456')
cursor = conn.cursor()

   ### 从数据库中取出数据。共取出以下数据字段：
- id，用户id，题目id，做题时长，日期，正确与否(0/1)，该用户对应的学校名，该题目对应的知识点id(goal)

### 再将取出的数据转成DataFrame格式，方便以后使用
- 关于排序：先对day进行排序，再对时间戳time排序。时间戳time是用户设备上的时间，故出错的更多，先用必对的day限制一下

In [3]:
cursor.execute('''
SELECT t1.*, t2.goal_id
FROM
(
    SELECT id, u_user, problem_id,duration, day, correct, u_school, from_unixtime(event_time/1000) AS time
    FROM events.frontend_event_orc
    WHERE day BETWEEN 20181001 AND 20181031
      AND event_key = 'clickLTTPSumbit'
      AND u_user != ''
      AND u_user IS NOT NULL
      AND u_school IS NOT NULL
      AND u_school in ('新民中学', '莒南七中', '临沂十一中', '南宁二中', '广东实验中学', '双语学校', '武陵中学', '吴有训实验学校', '武侯实验中学', '城关中学', '重庆三十七中', '东方中学', '胶南四中')
)t1
INNER JOIN
(
    SELECT id, goal_id
    FROM course.problem
)t2 ON t1.problem_id = t2.id
ORDER BY t1.day,t1.time ASC
''')

data = cursor.fetchall()
data = pd.DataFrame(data)
data.columns = ['id', 'uid', 'pid', 'dur', 'day', 'corr', 'sch_name', 'time', 'gid']
data['ts_pts'] = 0 

### 来看看我们取出来的数据，转换成DataFrame之后是啥样子吧！

In [4]:
data.tail()

,id,uid,pid,dur,day,corr,sch_name,time,gid,ts_pts
272339,5bd9cf044e4fa563c93e52e7,5bd9b32b37a1c6078d7ba59f,18db632a-75ca-11e7-b80b-e7218667489c,34909,20181031,1,新民中学,2018-10-31 23:49:13,9690ebf2-53f4-11e7-8610-ffb838397af0,0
272340,5bd9cf254e4fa563c93e8d63,5bd9b32b37a1c6078d7ba59f,6069ab72-75c3-11e7-b414-efbc79ae5142,20057,20181031,1,新民中学,2018-10-31 23:49:36,9690ebf2-53f4-11e7-8610-ffb838397af0,0
272341,5bd9cf494e4fa563c93ed0c9,5bd9b16971de5007ba60ace7,b9a921de-57e8-11e7-b9af-6ba7df29752e,16501,20181031,1,新民中学,2018-10-31 23:50:26,973a0106-53f4-11e7-8654-0718617ce756,0
272342,5bd9cf6d4e4fa563c93f1439,5bd9b16971de5007ba60ace7,5cd36af8-7b3a-11e7-aa1c-8705033a2960,36023,20181031,1,新民中学,2018-10-31 23:51:03,973a0106-53f4-11e7-8654-0718617ce756,0
272343,5bd9d0454e4fa563c9406c7d,5bd9b16971de5007ba60ace7,4427c682-7b46-11e7-8132-3fb936db8a73,212988,20181031,1,新民中学,2018-10-31 23:54:37,974564ce-53f4-11e7-8657-27a134c53335,0


$$compensate = \left (\frac{\mu_{g}}{\mu_{u}} - 1  \right ) \cdot b$$  
$$\Delta = \frac{\mu_g}{\mu_u} \cdot norm\_d(duration) + compensate,$$
where
<img src="https://images-cdn.shimo.im/zkpSlHD52FE0vDaz/%E5%B1%8F%E5%B9%95%E5%BF%AB%E7%85%A7_2018_06_11_%E4%B8%8B%E5%8D%882.45.06.png!thumbnail" style="width:500px"/>
(when 'corr' == 1)

In [5]:
def norm_duration(item):
    d = item['dur']
    d = max(2000, d)
    d = min(d, 600000)
    
    x = 2000/d
    x = x if item['corr'] == 1 else (1-x+2000/600000)
    return x ** 0.3

In [7]:
def cal_TrueSkill(data):
    user = defaultdict(ts.Rating)
    goal = defaultdict(ts.Rating)
    
    for i in data.iterrows():
        idx = i[0]
        item = i[1]
        # 对每一条迭代进来的记录，分别初始化uid和pid的能力值，然后用上式计算。
        u_player = user[item['uid']]
        g_player = goal[item['gid']]
        
        # 核心句，计算一次做题提交之后，trueskill的变化。根据user做题的对错情况，决定这个delta数值是正还是负。
        data.loc[idx,'ts_pts'] = 2*(item['corr'] - 0.5) * (g_player.mu / u_player.mu)*norm_duration(item)
        # 记录即时的goal与user值，是为了后面计算compensate
        data.loc[idx,'temp_g'] = g_player.mu
        data.loc[idx,'temp_u'] = u_player.mu
    
        if item['corr']:
            u_player,g_player = ts.rate_1vs1(u_player,g_player)
        else :
            g_player,u_player = ts.rate_1vs1(g_player,u_player)
            
        user[item['uid']] = u_player
        goal[item['gid']] = g_player
    return user,goal,data
# user/goal两个defaultdict分别存储某个user或某个goal的实时trueskill；data的‘ts_pts’列，存储该条记录对战过后，user应该发生的trueskill变化

In [8]:
def split_data(cores,data):
    split_num = np.linspace(0,len(data),5,dtype=int)
    data_seg = [data[split_num[j]:split_num[j+1]] for j in range(len(split_num)-1)]
    return data_seg

### 采用多进程方法进行计算每条记录的‘ts_pts’：
- 注：这里直接用了所有的cpu，在服务器上跑的时候好么？

In [9]:
cores = mps.cpu_count()
pool = mps.Pool(processes=cores)

r = []
data_seg = split_data(cores,data)
for i in data_seg:
    r.append(pool.apply_async(cal_TrueSkill, (i,))) 

pool.close()
pool.join()

res = [i.get() for i in r]
data = pd.concat([i[2] for i in res])

user_ts = defaultdict(ts.Rating)
goal_ts = defaultdict(ts.Rating)

for i in res:
    user_ts.update(i[0])
    goal_ts.update(i[1])

### 再看看我们的表格data 

In [10]:
data.tail(10)

,id,uid,pid,dur,day,corr,sch_name,time,gid,ts_pts,temp_g,temp_u
272334,5bd9cdb14e4fa563c93bb16d,5bd9b32b37a1c6078d7ba59f,b2d883fe-57e8-11e7-b80c-378e135d2a87,68256,20181031,1,新民中学,2018-10-31 23:43:37,9690ebf2-53f4-11e7-8610-ffb838397af0,0.306939,24.449324,27.623696
272335,5bd9cdc34e4fa563c93bd68e,5bd9b32b37a1c6078d7ba59f,6e2beefc-75c1-11e7-99f3-ff47a7d2619e,16692,20181031,1,新民中学,2018-10-31 23:43:54,9690ebf2-53f4-11e7-8610-ffb838397af0,0.464850,24.385892,27.757627
272336,5bd9ce024e4fa563c93c5c55,5bd9b32b37a1c6078d7ba59f,c89dfe26-75c5-11e7-8157-bfc977a98b79,53647,20181031,1,新民中学,2018-10-31 23:44:48,9690ebf2-53f4-11e7-8610-ffb838397af0,0.325172,24.324413,27.885293
272337,5bd9ce2c4e4fa563c93cb1e3,5bd99ca5d508f43a0f3a7d9b,b31f9640-57e8-11e7-b81c-eb297590a949,132919,20181031,1,新民中学,2018-10-31 23:45:32,96a3f21a-53f4-11e7-8616-5bf7296ab17e,0.268125,27.668910,29.301349
272338,5bd9cee34e4fa563c93e166d,5bd9b32b37a1c6078d7ba59f,fc290758-75c1-11e7-9cd7-ff2d34fe678f,45573,20181031,1,新民中学,2018-10-31 23:48:37,9690ebf2-53f4-11e7-8610-ffb838397af0,0.339159,24.264769,28.007178
272339,5bd9cf044e4fa563c93e52e7,5bd9b32b37a1c6078d7ba59f,18db632a-75ca-11e7-b80b-e7218667489c,34909,20181031,1,新民中学,2018-10-31 23:49:13,9690ebf2-53f4-11e7-8610-ffb838397af0,0.365000,24.206853,28.123716
272340,5bd9cf254e4fa563c93e8d63,5bd9b32b37a1c6078d7ba59f,6069ab72-75c3-11e7-b414-efbc79ae5142,20057,20181031,1,新民中学,2018-10-31 23:49:36,9690ebf2-53f4-11e7-8610-ffb838397af0,0.428316,24.150566,28.235299
272341,5bd9cf494e4fa563c93ed0c9,5bd9b16971de5007ba60ace7,b9a921de-57e8-11e7-b9af-6ba7df29752e,16501,20181031,1,新民中学,2018-10-31 23:50:26,973a0106-53f4-11e7-8654-0718617ce756,0.426807,25.700090,31.971243
272342,5bd9cf6d4e4fa563c93f1439,5bd9b16971de5007ba60ace7,5cd36af8-7b3a-11e7-aa1c-8705033a2960,36023,20181031,1,新民中学,2018-10-31 23:51:03,973a0106-53f4-11e7-8654-0718617ce756,0.336365,25.664946,32.052744
272343,5bd9d0454e4fa563c9406c7d,5bd9b16971de5007ba60ace7,4427c682-7b46-11e7-8132-3fb936db8a73,212988,20181031,1,新民中学,2018-10-31 23:54:37,974564ce-53f4-11e7-8657-27a134c53335,0.226215,29.488182,32.131372


### 有了未被弥补过的ts_pts之后，我们根据它来计算弥补值的系数b：

In [11]:
u_ts_max = max(list(map(lambda x:x.mu,list(user_ts.values()))))
u_ts_min = min(list(map(lambda x:x.mu,list(user_ts.values()))))
g_ts_max = max(list(map(lambda x:x.mu,list(goal_ts.values()))))
g_ts_min = min(list(map(lambda x:x.mu,list(goal_ts.values()))))
b1 = g_ts_min*(2000/600000)**0.3/(u_ts_max-g_ts_min)
b2 = g_ts_max/(g_ts_max-u_ts_min)
b = min(b1,b2)
b

0.08180668300189323

### 有了b之后，我们开始逐行计算弥补项compensate，以及加上compensate更新之后的ts_pts:
- 我们之所以可以这样算，是因为每一轮的得分最终不会影响到trueskill值的更新。即，你这次做错一道难题，你的得分最终是被compensate了，但其实你的trueskill的变化并没有被compensate（其实trueskill算法模块里肯定也帮你这样做了），所以不用重新计算，只需要计算compensate，然后加到ts_pts上即可。

In [12]:
def ts_update(data,b):
    for i,row in data.iterrows():
        compensate = (row['temp_g'] / row['temp_u'] - 1) * b
        data.loc[i,'compensate'] = compensate
        data.loc[i,'ts_pts'] = data.loc[i,'ts_pts'] + compensate
    return data

In [13]:
pool2 = mps.Pool(processes=cores)

r2 = []
data_seg2 = split_data(cores,data)
for i in data_seg2:
    r2.append(pool2.apply_async(ts_update,args=(i,b,)))

pool2.close()
pool2.join()

res2 = [i.get() for i in r2]
data = pd.concat([i for i in res2])

### 看一下compensate之后的data：

In [15]:
sum((data['corr'] - 0.5) * data['ts_pts'] <0)    # 确认一下，没人做对了题得分为负，也没人做错了题得分为正

0

In [16]:
data.tail()

,id,uid,pid,dur,day,corr,sch_name,time,gid,ts_pts,temp_g,temp_u,compensate
272339,5bd9cf044e4fa563c93e52e7,5bd9b32b37a1c6078d7ba59f,18db632a-75ca-11e7-b80b-e7218667489c,34909,20181031,1,新民中学,2018-10-31 23:49:13,9690ebf2-53f4-11e7-8610-ffb838397af0,0.353607,24.206853,28.123716,-0.011393
272340,5bd9cf254e4fa563c93e8d63,5bd9b32b37a1c6078d7ba59f,6069ab72-75c3-11e7-b414-efbc79ae5142,20057,20181031,1,新民中学,2018-10-31 23:49:36,9690ebf2-53f4-11e7-8610-ffb838397af0,0.416481,24.150566,28.235299,-0.011835
272341,5bd9cf494e4fa563c93ed0c9,5bd9b16971de5007ba60ace7,b9a921de-57e8-11e7-b9af-6ba7df29752e,16501,20181031,1,新民中学,2018-10-31 23:50:26,973a0106-53f4-11e7-8654-0718617ce756,0.410760,25.700090,31.971243,-0.016046
272342,5bd9cf6d4e4fa563c93f1439,5bd9b16971de5007ba60ace7,5cd36af8-7b3a-11e7-aa1c-8705033a2960,36023,20181031,1,新民中学,2018-10-31 23:51:03,973a0106-53f4-11e7-8654-0718617ce756,0.320061,25.664946,32.052744,-0.016303
272343,5bd9d0454e4fa563c9406c7d,5bd9b16971de5007ba60ace7,4427c682-7b46-11e7-8132-3fb936db8a73,212988,20181031,1,新民中学,2018-10-31 23:54:37,974564ce-53f4-11e7-8657-27a134c53335,0.219485,29.488182,32.131372,-0.006730


### 计算下列内容：
- 1.某学校总做题量；                                **final_res['做题量']**
- 2.参与进来的用户总数；                             **final_res['用户总数']**
- 3.某学校每位用户平均做题量；                        **final_res['人均题量']**
- 4.利用F1积分法等方法得到的合理的学校总分；            **final_res['总分Func']**
- 5.学校学生的平均trueskill；                      **final_res['人均TrueSkill']**
- 6.每个学校涉及到的知识点goal总数；                 **final_res['知识点数']**                      
- 7.本学校拿到前三名的知识点goal数量；

In [17]:
final_res = pd.DataFrame(dict())

In [18]:
final_res['做题量'] = data.groupby('sch_name').apply(lambda x: len(x))

In [19]:
uid_count = [len(x) for x in data.groupby('sch_name').uid.unique()]

final_res['用户总数'] = uid_count
final_res['人均题量'] = final_res['做题量'].values / uid_count

In [20]:
sch_user_ts = data[['uid','sch_name']]
sch_user_ts = sch_user_ts.drop_duplicates()
sch_user_ts['trueskill'] = sch_user_ts.uid.apply(lambda x:user_ts[x].mu)  
# 因为我们无法把duration的计算加入trueskill公式中，因而在计算trueskill的只好忽略掉它。但涉及到最后排名时，duration会被用上。
final_res['人均TrueSkill'] = sch_user_ts.groupby('sch_name').apply(lambda x:x['trueskill'].mean())

In [21]:
final_res['知识点数'] = data.groupby('sch_name').apply(lambda x:len(x.gid.unique()))

In [32]:
# 每个知识点下每个学校得分排名：
F1_score = {1:25, 2:18, 3:15, 4:12, 5:10, 6:8, 7:6, 8:4, 9:2, 10:1}

rs = data.groupby(['gid','sch_name']).apply(lambda x: sum(x.ts_pts))
rs = pd.DataFrame(rs).reset_index()
rs.columns = ['gid', 'sch_name', 'pt']
rs['rank_int'] = rs[['gid', 'pt']].groupby(['gid']).rank(ascending=0)
rs['rank_pct'] = rs[['gid','pt']].groupby('gid').rank(ascending=True,method='max',pct=True)
rs['Func_pt'] = rs['rank_pct'].apply(lambda x:pow((x**8),1/3)*25)
rs['F1_pt'] = rs['rank_int'].apply(lambda x:F1_score.get(x,0))
rs.head(10)

,gid,sch_name,pt,rank_int,rank_pct,Func_pt,F1_pt
0,006f814e-b329-11e7-aab6-b3208c44f7d9,临沂十一中,-3.488822,3.0,0.6,6.402408,15
1,006f814e-b329-11e7-aab6-b3208c44f7d9,南宁二中,-0.176061,1.0,1.0,25.000000,25
2,006f814e-b329-11e7-aab6-b3208c44f7d9,新民中学,-1.588062,2.0,0.8,13.788382,18
3,006f814e-b329-11e7-aab6-b3208c44f7d9,武陵中学,-4.187023,4.0,0.4,2.171534,12
4,006f814e-b329-11e7-aab6-b3208c44f7d9,莒南七中,-9.963761,5.0,0.2,0.341995,10
5,01f90096-7123-11e7-b3a9-2b20ad1b4a83,临沂十一中,-5.351820,5.0,0.2,0.341995,10
6,01f90096-7123-11e7-b3a9-2b20ad1b4a83,南宁二中,-0.973616,3.0,0.6,6.402408,15
7,01f90096-7123-11e7-b3a9-2b20ad1b4a83,新民中学,3.106247,1.0,1.0,25.000000,25
8,01f90096-7123-11e7-b3a9-2b20ad1b4a83,武陵中学,-3.815430,4.0,0.4,2.171534,12
9,01f90096-7123-11e7-b3a9-2b20ad1b4a83,莒南七中,-0.602257,2.0,0.8,13.788382,18


In [33]:
final_res['总分Func'] = rs.groupby('sch_name').apply(lambda x:sum(x['Func_pt']))
final_res['总分F1'] = rs.groupby('sch_name').apply(lambda x:sum(x['F1_pt']))
final_res['平均顺位'] = rs.groupby('sch_name').apply(lambda x:x['rank_int'].mean())
final_res['得到第一的次数'] = rs.groupby('sch_name').apply(lambda x:sum(x['rank_int'] == 1))
final_res['得到前二的次数'] = rs.groupby('sch_name').apply(lambda x:sum(x['rank_int'] < 2.1))
final_res['得到前三的次数'] = rs.groupby('sch_name').apply(lambda x:sum(x['rank_int'] < 3.1))
final_res['6名开外次数'] = rs.groupby('sch_name').apply(lambda x:sum(x['rank_int'] > 6.9))

In [34]:
final_res.sort_values(by='总分Func',ascending=False)
# 注意吴有训和双语，吴有训在Func中反超了双语，就是因为他们前二名的次数明显多于双语学校。可见Func法对排名靠前的学校的得分有较好的区分度

,做题量,用户总数,人均题量,人均TrueSkill,知识点数,总分Func,平均顺位,得到第一的次数,得到前二的次数,得到前三的次数,6名开外次数,总分F1
sch_name,,,,,,,,,,,,
莒南七中,93206,608,153.299342,26.972697,934,10924.523746,3.652034,312,447,533,178,14445
新民中学,14736,348,42.344828,27.747678,845,7639.991891,3.879290,128,256,378,101,11627
武陵中学,70310,601,116.988353,26.448467,911,7571.238877,4.439078,154,311,425,222,11745
临沂十一中,56678,808,70.146040,26.790664,854,6257.141674,4.567916,79,206,385,202,10358
城关中学,12764,482,26.481328,27.947369,568,5623.255261,4.242958,106,186,253,132,7529
南宁二中,5017,154,32.577922,27.791836,604,5367.806548,4.228477,67,162,249,99,7733
双语学校,11149,347,32.129683,27.030771,450,4001.061359,4.504444,49,112,184,99,5517
吴有训实验学校,3412,113,30.194690,27.146092,403,3646.666496,4.543424,58,108,156,94,4971
广东实验中学,1705,67,25.447761,27.797391,380,3124.020405,4.657895,36,95,137,82,4511


### 以吴有训实验学校为例，计算该校每个用户对于学校总成绩的贡献值：
- 以百分数形式表现

In [35]:
wu = data[data['sch_name'] == '吴有训实验学校']
ts_wu = pd.DataFrame(dict())
ts_wu['ts'] = wu.groupby('uid').apply(lambda x:sum(x['ts_pts']))
ts_wu['rank'] = ts_wu.rank(method='min',ascending=True,pct=True)

In [36]:
ts_wu.sort_values(by='ts',ascending=False)

,ts,rank
uid,,
5b9a6afd054352062d54c1d2,17.182649,1.000000
5b9c74a620867a22e6a21ead,13.396002,0.991150
5bcab216aa863e07e36d0243,12.923602,0.982301
5b99d0ce3ba133062b33e55c,11.567244,0.973451
59f040cb9dcce33008c725c3,10.072510,0.964602
59f0950b3d3672091295c132,9.658368,0.955752
5b8b908c6ce1ce06318ec217,8.474387,0.946903
5b8b91126ce1ce06318ec25d,6.868173,0.938053
5b99bbbdf9ea50062cf1e077,6.582848,0.929204
